<img src="images/econ140R_logo.png" width="200" />

<h1>ECON 140R Class 01_attend</h1>

Let us examine scores in the Fall 2023 version of ECON 140. To proceed, we will load up `tidyverse` because it is often useful, and `haven` because it helps us read datasets in Stata format.

In [ ]:
library(tidyverse)
library(haven)

The `haven` command `read_dta()` pulls in a data frame from a Stata .dta file, and the __R__ command `head()` offers a look at the top of the data frame:

In [ ]:
econ140_f23 <- read_dta("data/econ140_f23_attendance_blinded.dta")

In [ ]:
head(econ140_f23, n = 5)

Here are the variables in the dataset:
* `attend` self-reported frequency of in-person attendance in the first half of the course
    * 1 = "attended in person most or all of the time"
    * 2 = "a majority of the time"
    * 3 = "about half the time"
    * 4 = "some of the time"
    * 5 = "rarely or never"
* `project` the score on the project in percent
* `imidterm` the score on the in-class midterm exam in percent
* `omidterm` the score on the online 2nd midterm exam in percent
* `final` the score on the final exam in percent
* `total` total weighted points earned in percent
* `plagquiz` the score on the plagiarism+ quiz in percent
* `pollev` the score on Poll Everywhere participation in percent
* `psets` the score on the problem sets in percent

First up, let's look at a histogram of the total weighted points. Here is some simple code that does that:

In [ ]:
hist(econ140_f23$total, 
     breaks = c(0:100),
     xlim = c(40,100),
     main = "ECON 140 F23 Total Score",
     xlab = "ECON 140 F23 Total Score in percent"
     )

<h2>The elephant (not?) in the room: In-person attendance</h2>

A natural question we'd ideally like to ask is how in-person attendance causally affects students' outcomes. In the second midterm conducted online in Fall 2023, I asked ECON 140 students to self-report their frequency of in-person attendance in the first half of the course. Students received a point for any answer, and they could also refuse to answer and still receive a point. A small number of students refused to answer.

There are several minor limitations associated with the way the data was collected. Did students recall correctly enough? Were they likely to report truthfully?

But by far the bigger issue here is that <b>students were choosing how much to attend in-person</b>. Synchronous attendance was part of the grade, but in-person attendance was not compulsory, nor was it rewarded with extra credit.

In ECON 140, it will be useful to think separately about a <b>causal effect</b> of a treatment variable of interest, $X$, on an outcome variable of interest, $Y$, and a <b>selection effect</b>.  

The <b>causal effect</b> of $X$ on $Y$ is relevant for policy; it is what we would see happen to $Y$ if a policy applied the treatment of $X$ to people and they complied.  Think of a vaccination against a disease as $X$ and health or mortality as the outcome $Y$. (Note that $X$ could have no effect or even be bad for $Y$! This is why we have the Food and Drug Administration.) 

The <b>selection effect</b> of $X$ on $Y$ derives from other variables that are present and shifting both $X$ and thus $Y$ but which are <i>omitted</i>, causing bias. Think of a third variable like political affiliation, $Z$, which may affect the uptake of a vaccination $X$ and might also independently affect $Y$.

Here, when $X$ is in-person attendance and $Y$ is the total score, what are our priors about causal effects? Does the quality of the lecture hall matter? What about class size? Quality of campus wifi?

What are our priors about selection effects? What other variables prompt students not to attend? (You could think of variables that prompt them to attend also, but I find the other question easier to answer.)

<hr>

To begin, let's look at attendance self reports. The code below looks at broad tabulations in raw counts, then prints the total count of students, and then the shares:

In [ ]:
attend_freq <- table(econ140_f23$attend)
attend_freq

In [ ]:
sum(attend_freq)

In [ ]:
attend_freq/sum(attend_freq)

<hr>

Now let's look at total scores. Here is the unconditional mean across all students:

In [ ]:
mean(econ140_f23$total) 

And here is one way to look at the mean conditional on a value for `attend`, using `subset()`.

In [ ]:
mean(subset(econ140_f23, attend == 1)$total)

In [ ]:
mean(subset(econ140_f23, attend == 2)$total)

In [ ]:
mean(subset(econ140_f23, attend == 3)$total)

In [ ]:
mean(subset(econ140_f23, attend == 4)$total)

In [ ]:
mean(subset(econ140_f23, attend == 5)$total)

The plot thickens!

We see some non-monotonicities here: As `attend` rises across the 5 responses indicating increasingly less in-person attendance, we see a small drop, then an increase, then another small drop, then zero effect.

There are many ways to proceed. One really valuable view is to use <b>ordinary least squares</b>. We will estimate this model:

$$
total_i = \alpha + \beta \ attend_i + \epsilon_i
$$

where $\alpha$ is a hypothetical total score when $attend = 0$, which never actually happens in these data; $\epsilon$ is a white-noise error term; and $\beta$ is a composite of any causal effects plus any selection effects of $attend$ on $total$.

It helps if we can visualize what we are doing, but sometimes visualizations are rough.

In [ ]:
plot(econ140_f23$attend, econ140_f23$total)

Ugh. Sort of a "The Matrix" movie poster except much sparser.

In social science, it is common for outcome and treatment variables to be lumpy and discrete rather than continuous. In cases like these, scatterplots might just create more confusion rather than help us visualize relationships. 

But it turns out that __R__ has a built-in function `jitter()`, [documented here](https://search.r-project.org/R/refmans/base/html/jitter.html) which can inject randomness into observations. Below is some code that does it, with the `amount` parameter set to 1. It's good form to `set.seed()` to something, perhaps today's date, so that you can reproduce what you see.

In [ ]:
set.seed(20240829)

plot(jitter(econ140_f23$attend, amount = 1), 
     jitter(econ140_f23$total, amount = 1))

Still not exactly clear, but at least it's less offensive to the eye.

Here is the tool that estimates <b>ordinary least squares</b> for us: `lm()` in __R__, which stands for "linear models."

In [ ]:
reg1 <- lm(total ~ attend, 
           data = econ140_f23)
summary(reg1)

Each unit step of increasingly less frequent in-person attendance here is associated with an increase in the total score of 0.6704 point, and the <b>standard error</b> of that coefficient estimate $\hat{\beta}$ is 0.2467. Its $t$-stat is 2.717, and that is larger than the critical value of about 2, meaning that the coefficient estimate is more than 2 of its standard errors different from zero. 

<b>We reject the null hypothesis that there is no effect of attendance on the total ECON 140 score.</b> The coefficient is positive, and because an increase in `attend` means a reduction in attendance frequency, we conclude that in ECON 140 in Fall 2023, <i>less attendance was associated with higher total scores, and that effect was statistically significantly different from zero.</i> 

Are you shocked?

<hr>

Once we have estimated the model, we can extract the prediction of the outcome variable, usually denoted as $\hat{Y}$. The extra code below is necessary when there were missing values in the data frame.

In [ ]:
econ140_f23$total_hat <- predict(reg1, newdata = econ140_f23)

And now here is the picture with the least-squares trendline overlaid in blue:

In [ ]:
plot(jitter(econ140_f23$attend, amount = 1), 
     jitter(econ140_f23$total, amount = 1))
lines(econ140_f23$attend,
      econ140_f23$total_hat,
      col = "blue")

Not exactly convincing, is it? It sort of looks like Dad's mouth after eating blueberries, and he's mildly ... amused, maybe? 

Thoughts?

My takeaways from this are:
* Applied microeconomics can produce pretty unimpressive graphics!
* This might be why economists love regression tables more. In the regression, the statistics are showing us something that the eye can't see in the picture
* There probably is <b>selection</b> and <b>causality</b> in the association between in-person attendance and academic outcomes
* Many observational studies will show similar results
* Much of the magic of econometrics is <i>explaining what you see using these perspectives</i> and thinking about how you could better see what you want to

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>